In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

label_methods = [
    "random",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]
search_methods = [
    'clusters',
    'markov',
    'lgmm',
    'wkmeans',
    'mapie',
    'causal',
]

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='random', search_type='lgmm', search_subtype='',
        n_models=1, debug=False,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None
if not study:
    n_trials = 5000
else:
    n_trials = len(study.trials) + 5000

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 1/5000 score=9.768482 avg=64.40s mem=463.48MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 2/5000 score=9.768482 avg=35.86s mem=519.02MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 3/5000 score=9.768482 avg=24.80s mem=522.10MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 4/5000 score=11.993219 avg=23.96s mem=567.24MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 5/5000 score=11.993219 avg=19.57s mem=595.84MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 6/5000 score=11.993219 avg=19.27s mem=524.07MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 7/5000 score=11.993219 avg=18.74s mem=528.69MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 8/5000 score=11.993219 avg=17.10s mem=577.94MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 9/5000 score=11.993219 avg=15.94s mem=538.50MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 10/5000 score=15.253036 avg=17.98s mem=734.39MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 11/5000 score=15.253036 avg=16.42s mem=735.39MB 
